In [ ]:
# https://www.tensorflow.org/tensorboard/tensorboard_projector_plugin
#https://branyang.gitbooks.io/tfdocs/content/get_started/embedding_viz.html
# https://projector.tensorflow.org/
#https://www.youtube.com/watch?v=e2w-kOVHNQ4
# https://towardsdatascience.com/taking-the-tensorboard-embedding-projector-to-the-next-level-bde53deb6bb7
#https://medium.com/looka-engineering/how-to-visualize-feature-vectors-with-sprites-and-tensorflows-tensorboard-3950ca1fb2c7
#https://github.com/AndrewBMartin/tensorboard_sprites/blob/master/tensorboard_sprites.ipynb

In [ ]:
%load_ext tensorboard
import csv
import numpy as np
import tensorflow as tf
from PIL import Image

In [ ]:
embeddings = tf.keras.models.Model( 
    inputs=model.inputs, 
    outputs=model.layers[-1].output 
)

In [ ]:
def get_img(img_path):
    img = tf.io.read_file(img_path)
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # resize the image to the desired size for your model
    img = tf.image.resize_with_pad(img, 100, 100)
    return img
# Generate embeddings
images_pil = []
images_embeddings = []
labels = []
for x in raw_val_ds.take(1500): 
    img_path = x[0]
    img_tf = get_img(img_path)
    # Save both tf image for prediction and PIL image for sprite
    img_pil = Image.open(img_path.numpy()).resize((100, 100))
    img_embedding = embeddings(tf.expand_dims(img_tf, axis=0))
    images_embeddings.append(img_embedding.numpy()[0])
    images_pil.append(img_pil)
    # Assuming your output data is directly the label
    label = x[1] 
    labels.append(label)

In [ ]:
with open(f’{LOG_DIR}/embeddings/feature_vecs.tsv’, ‘w’) as fw:
    csv_writer = csv.writer(fw, delimiter=’\t’)
    csv_writer.writerows(images_embeddings)

In [ ]:
one_square_size = int(np.ceil(np.sqrt(len(images_embeddings))))
master_width = 100 * one_square_size
master_height = 100 * one_square_size
spriteimage = Image.new(
    mode=’RGBA’,
    size=(master_width, master_height),
    color=(0,0,0,0) # fully transparent
)
for count, image in enumerate(images_pil):
    div, mod = divmod(count, one_square_size)
    h_loc = 100 * div
    w_loc = 100 * mod
    spriteimage.paste(image, (w_loc, h_loc))
spriteimage.convert(“RGB”).save(f’{LOG_DIR}/embeddings/sprite.jpg’, transparency=0)

In [ ]:
with open(f’{LOG_DIR}/embeddings/metadata.tsv’, ‘w’) as file: 
    for label in labels:
        file.write(f”{label}\n”)

In [ ]:
%tensorboard — logdir LOG_DIR